In [1]:
# load necessary packages; make sure install them first
using BenchmarkTools, CSV, DataFrames, DelimitedFiles, Distributions
using Ipopt, LinearAlgebra, MathOptInterface, MixedModels, NLopt
using PrettyTables, Random, RCall

const MOI = MathOptInterface

MathOptInterface

In [17]:
#import Pkg; Pkg.add("RCall")

   Resolving package versions...
   Installed CategoricalArrays ─ v0.10.6
    Updating `~/.julia/environments/v1.7/Project.toml`
  [6f49c342] + RCall v0.13.13
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [324d7699] + CategoricalArrays v0.10.6
  [6f49c342] + RCall v0.13.13
  [1b915085] + WinReg v0.3.1
Precompiling project...
  ✓ WinReg
  ✓ CategoricalArrays
  ✓ RCall
  3 dependencies successfully precompiled in 7 seconds (308 already precompiled)


### **Q1. (Optional, 30 bonus pts) Derivatives**

1. Prove the following derivatives:

- $\nabla_\boldsymbol{\beta} \ell_i (\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = \mathbf{X_i}^{T} \mathbf{\Omega_i}^{-1}\mathbf{r_i}$,
- $\nabla_{\sigma^2} \ell_i (\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = -\frac{1}{2} tr(\mathbf{\Omega_i}^{-1}) + \frac{1}{2}\mathbf{r_i^{T}\Omega_i^{-2}r_i}$,

### **Q2. (20 pts) Objective and gradient evaluator for a single datum**

We expand the code from HW3 to evaluate both objective and gradient. I provide my code for HW3 below as a starting point. You do not have to use this code. If your come up faster code, that's even better.

#### **Simplification of:** 
$\nabla_\boldsymbol{\beta} \ell_i (\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = \mathbf{X_i}^{T} \mathbf{\Omega_i}^{-1}\mathbf{r_i}$ 

We can use the Sherman Woodbury formula and a Cholesky decomposition to simply $M = (\sigma^2I + Z_iLL^{T}Z_i^{T})^{-1}$, resulting in: $M^{-1} = \frac{1}{\sigma^2}I - \frac{1}{\sigma^2}Z_iL(AA^{T})^{-1}L^{T}Z_i^{T}$

$(AA^{T})^{-1}$ is the result of the Cholesky decomposition. A is a lower triangular matrix, and A' is an upper triangular matrix. However, in the code below, we extract the upper triangular matrix and store it as 'V' (not explicitly stored in the code as V, but I will write is as such in the math below so the results in the code are more clear). 

\begin{align}
X_i^{T} \Omega_i r_i &= X_i^{T} (\sigma^2I + Z_iLL^{T}Z_i^{T})^{-1}(y_i - X_i\beta) \\
&= X_i^{T}\Big(\frac{1}{\sigma^2}I - \frac{1}{\sigma^2}Z_iL(V^{T}V)^{-1}L^{T}Z_i^{T}\Big)(y_i - X_i\beta) \\
&= \frac{1}{\sigma^2} (X_i^{T}y_i - X_i^{T}Z_iLV^{-1}(V^{T})^{-1}L^{T}Z_i^{T}y_i - X_i^{T}X_i\beta + X_i^{T}Z_iV^{-1}(V^{T})^{-1}L^{T}Z_i^{T}X_i\beta) \\
&= \frac{1}{\sigma^2} (X_i^{T}y_i - X_i{T}X_i\beta - X_i^{T}Z_iLV^{-1}(V^{T})^{-1}(Z_i^{T}y_i - Z_i^{T}X_i\beta))
\end{align}


In [10]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # arrays for holding gradient
    ∇β         :: Vector{T}
    ∇σ²        :: Vector{T}
    ∇Σ         :: Matrix{T}    
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    xty        :: Vector{T}
    zty        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    storage_q2 :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    xtz        :: Matrix{T} # added by me
    storage_qq :: Matrix{T}
    storage_qp :: Matrix{T}
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
    ) where T <: AbstractFloat
    n, p, q    = size(X, 1), size(X, 2), size(Z, 2)    
    ∇β         = Vector{T}(undef, p)
    ∇σ²        = Vector{T}(undef, 1)
    ∇Σ         = Matrix{T}(undef, q, q)    
    yty        = abs2(norm(y))
    xty        = transpose(X) * y
    zty        = transpose(Z) * y    
    storage_p  = Vector{T}(undef, p)
    storage_q  = Vector{T}(undef, q)
    storage_q2 = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    xtz        = transpose(X) * Z # added by me
    storage_qq = similar(ztz)
    storage_qp = similar(ztx)
    LmmObs(y, X, Z, ∇β, ∇σ², ∇Σ, 
        yty, xty, zty, storage_p, storage_q,
        storage_q2, xtx, ztx, ztz, xtz, storage_qq, storage_qp)
end

"""
    logl!(obs::LmmObs, β, L, σ², needgrad=false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `L`, 
and `σ²`. If `needgrad==true`, then `obs.∇β`, `obs.∇Σ`, and `obs.σ² are filled 
with the corresponding gradient.
"""
function logl!(
        obs      :: LmmObs{T}, 
        β        :: Vector{T}, 
        L        :: Matrix{T}, 
        σ²       :: T,
        needgrad :: Bool = true
    ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    ####################
    # Evaluate objective
    ####################    
    # form the q-by-q matrix: M = σ² * I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq) 
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq) 
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    # cholesky on M = σ² * I + Lt Zt Z L
    LAPACK.potrf!('U', obs.storage_qq) # extract A' = V from cholesky on M 
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.storage_q, obs.zty)) # z'y - z'xβ
    BLAS.trmv!('L', 'T', 'N', L, obs.storage_q)    # L'(z'y - z'xβ)
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, obs.storage_q) # V'^{-1} L'(z'y - z'xβ)
    # l2 norm of residual vector
    copy!(obs.storage_p, obs.xty)
    rtr  = obs.yty +
        dot(β, BLAS.gemv!('N', T(1), obs.xtx, β, T(-2), obs.storage_p))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (rtr - qf) / σ² 
    logl /= -2
    ###################
    # Evaluate gradient
    ###################    
    if needgrad
        # TODO: fill ∇β, ∇L, ∇σ² by gradients
        #sleep(1e-3) # pretend this step takes 1ms
        
        ####### gradient wrt β #######
        
        # term 1 xty - xtxβ
        
        copy!(obs.∇β, obs.xty) # ∇β now contains xty
        BLAS.gemv!('N', AbstractFloat(-1), obs.xtx, β, AbstractFloat(1), obs.∇β) # overwriting ∇β with x'y - x'x β
        
        # term 2 xtzL(AA')^{-1}L'(zty - ztxβ) 
        
        BLAS.trsv!('U', 'N', 'N', obs.storage_qq, obs.storage_q) 
        # cholesky extracted for M was upper 
        # this gets us A^{-1} L'(zty - ztxβ)
        
        BLAS.trmv!('L', 'N', 'N', L, obs.storage_q)
        # this gets us (A')^{-1}A^{-1} L'(zty - ztxβ)
        
        # combine the two terms
        
        BLAS.gemv!('N', AbstractFloat(-1)/σ², obs.xtz, obs.storage_q, AbstractFloat(1)/σ², obs.∇β)
        # subtracting terms 1 and 2 and dividing by σ²
        
        ####### gradient wrt σ² #######
        
        
        
        
        
        
        
        
        
        
        
        
    end    
    ###################
    # Return
    ###################        
    return logl 
end

logl!

In [11]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### **2.1  Correctness**

In [13]:
@show logl = logl!(obs, β, L, σ², true)
@show obs.∇β
@show obs.∇σ²
@show obs.∇Σ;

logl = logl!(obs, β, L, σ², true) = -3256.179335805826
obs.∇β = [0.2669810805714521, 41.61418337067322, -34.34664962312688, 36.108985107075306, 27.913948208793148]
obs.∇σ² = [2.5353974416e-314]
obs.∇Σ = [2.5271783236e-314 2.1792481733e-314 2.5271792643e-314; 2.5271783315e-314 2.1792489955e-314 2.2828103643e-314; 2.179249786e-314 2.2828103643e-314 2.2828103643e-314]


In [14]:
@assert abs(logl - (-3256.1793358058258)) < 1e-4
@assert norm(obs.∇β - [0.26698108057144054, 41.61418337067327, 
        -34.34664962312689, 36.10898510707527, 27.913948208793144]) < 1e-4
# @assert norm(obs.∇Σ - 
#     [-0.9464482950697888 0.057792444809492895 -0.30244127639188767; 
#         0.057792444809492895 -1.00087164917123 0.2845116557144694; 
#         -0.30244127639188767 0.2845116557144694 1.170040927259726]) < 1e-4
#@assert abs(obs.∇σ²[1] - (1.6283715138412163)) < 1e-4

### **2.2  Efficiency**

Benchmark for evaluating the objective function only. This is what we did in HW3.

In [141]:
@benchmark logl!($obs, $β, $L, $σ², false)

BenchmarkTools.Trial: 10000 samples with 74 evaluations.
 Range (min … max):  806.419 ns …   3.819 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     836.858 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   974.620 ns ± 267.583 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁█▅▃  ▁ ▅▃▂▁▁▁ ▁▁ ▁▁▁  ▁         ▁                            ▁
  ███████▆██████████████▇█▇▇██▇█▇█████▇▇▇█▇▇▇▇▆▆▆▇▆▆▅▅▄▅▆▄▅▄▅▅▅ █
  806 ns        Histogram: log(frequency) by time          2 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

Benchmark for objective + gradient evaluation.

In [143]:
bm_objgrad = @benchmark logl!($obs, $β, $L, $σ², true)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.177 μs …  14.162 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.229 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.464 μs ± 569.190 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▅▃▅▄▃▃▂▃▂ ▃ ▃ ▂ ▁ ▂▂  ▂                                   ▁
  █████████████▇█▇███▇███▇█▆▇▅▆▅▆▇▇▆▆▆▆▅▅▆▆▅▅▆▆▆▄▆▅▅▄▄▄▄▅▂▂▄▃ █
  1.18 μs      Histogram: log(frequency) by time      3.78 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [144]:
#  The points you will get are
clamp(10 / (median(bm_objgrad).time / 1e3) * 10, 0, 10)

10.0

# **Testing my Code Line by Line**

3×3 Matrix{Float64}:
  45.1902    4.72605   4.40943
 213.571    44.2074    2.32431
 199.263   123.591    44.1539